In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [23]:
data = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")

In [24]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
data.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [25]:
# function for column standardisation
def standard(series):
    m = series.mean()
    sd = series.std()
    series = (series - m )/sd
    return series

In [28]:
#standardizing each column
features=['petal_length','petal_width','sepal_length','sepal_width']
data[features]=data[features].apply(standard)

In [29]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,-0.897674,1.028611,-1.336794,-1.308593,setosa
1,-1.139200,-0.124540,-1.336794,-1.308593,setosa
2,-1.380727,0.336720,-1.393470,-1.308593,setosa
3,-1.501490,0.106090,-1.280118,-1.308593,setosa
4,-1.018437,1.259242,-1.336794,-1.308593,setosa


In [30]:
data.describe() #prints descriptive statistics of all numerical features present in our data

,sepal_length,sepal_width,petal_length,petal_width
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-1.457168e-15,-1.722511e-15,-2.043551e-15,-9.843977e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.863780e+00,-2.430844e+00,-1.563497e+00,-1.439627e+00
25%,-8.976739e-01,-5.858010e-01,-1.223442e+00,-1.177559e+00
50%,-5.233076e-02,-1.245404e-01,3.351431e-01,1.327811e-01
75%,6.722490e-01,5.673506e-01,7.602119e-01,7.879511e-01
max,2.483699e+00,3.104284e+00,1.780377e+00,1.705189e+00


In [7]:
data.shape

(150, 5)

In [8]:
data_train = data.sample(120,random_state=17) # dividing data into d_train and d_test randomly
index_list = data_train.index.tolist()
index_test = []
for i in range(150):
    if i not in index_list:
        index_test.append(i)
data_test = data.iloc[index_test]

In [9]:
data_train.head(1)

,sepal_length,sepal_width,petal_length,petal_width,species
16,-0.535384,1.951133,-1.39347,-1.046525,setosa


In [64]:
data_test.tail(3)

,sepal_length,sepal_width,petal_length,petal_width,species
138,0.189196,-0.12454,0.590184,0.787951,virginica
139,1.276066,0.10609,0.930239,1.181053,virginica
143,1.155302,0.33672,1.213619,1.443121,virginica


In [31]:
X_train = data_train[["sepal_length","sepal_width","petal_length","petal_width"]]
Y_train = data_train["species"]
X_test = data_test[["sepal_length","sepal_width","petal_length","petal_width"]]
Y_test = data_test["species"]

In [32]:
X_train.iloc[1,2]

0.4201568543133208

In [33]:
def dist(x,X_train,dist_type = "euc"): #return the euclidean distance betweeen x and each point of X_train
    n_rows , n_cols = X_train.shape
    #train_n = X_train.values #convert X_train which is a DataFrame to numpy nd array so that we can index both rows and column by integer value
    euc = []
    for row in range(n_rows):
        sqr = 0 
        for col in range(n_cols):
            sqr += (x[col] - X_train.iloc[row,col])**2
        sqrt = sqr**(1/2)
        #print(sqrt)
        euc.append(sqrt)
    return euc #return the euclidean distance betweeen x(single data point of X_test) and each point of X_train

In [75]:
def k_nn(X_train,Y_train,X_test,Y_test,dist_type="euc",neighbor=5,task="R"):
    n_rows , n_cols = X_train.shape
    n_rows_test , n_cols_test = X_test.shape
    predicted_y = []
    if dist_type == "euc":
        for i in index_test: #select data points from X_train one by one
            x = X_test.loc[i] #selected single data point of  X_test
            dis = dist(x,X_train,dist_type="euc") #dis contains the euclidean distance between x and each point of training data
            temp_data = X_train.copy() #creates copy of Training data and this copy is created for each data point of X_test 
            """adding a new column or series for a single data point of X_train which holds the value of
            euclidean distance between x and each point of training data""" 
            temp_data["Distance"] = dis 
            #adding the column or series of Y_train
            temp_data["Y"] = Y_train
            #sorting on the basis of distance in ascending order as train_points which are closer to x should be selected
            sorted_X = temp_data.sort_values(by="Distance",inplace=False,ascending=True) 
            #picking up only upper "N" Neighbor
            n_values = sorted_X.iloc[0:neighbor,:]
            if task=="R": #if K-NN is used for Regression
                pred = n_values["Y"].median()
                predicted_y.append(pred)
            else: #if K-NN is used for Classification
                #doing majority vote out of N-neighbor
                series = n_values["Y"].value_counts()
                predicted_y.append(list(series.index)[0]) #adding predicted class to the list predicted_y 
    return predicted_y

In [76]:
#using function k-NN for prediction
predd = k_nn(X_train,Y_train,X_test,Y_test,dist_type="euc",neighbor=5,task="C") 

In [77]:
count = 0
total_test = len(Y_test)
for i in range(len(predd)):
    if predd[i] == Y_test.iloc[i]:
        count += 1
accuracy = (count/total_test)*100
print("The accuracy of our model which is implemented from scratch is {}%".format(accuracy))

The accuracy of our model which is implemented from scratch is 93.33333333333333%
